In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd

In [2]:
TEST_FOLDER = "/root/shared/ai-journey-2019/data/check/"

In [3]:
tests_list = glob.glob(os.path.join(TEST_FOLDER, "*.json"))

In [4]:
tests_dfs = []

for test_path in tests_list:
    test_ = json.load(open(test_path))
    tests_dfs.append(pd.DataFrame.from_dict(test_['tasks']))

In [5]:
df_tests = pd.concat(tests_dfs)

# Automatic task type classification:

### Data preparation:

In [6]:
df_classification = df_tests[["id"]].astype(int)

In [7]:
# add question type
df_classification["type"] = df_tests.question.apply(lambda x: x['type'])
# add max_length
# df_classification["max_length"] = df_tests.question.apply(lambda x: x['max_length'])

# add raw text:
df_classification["text"] =  df_tests.text

# add meta language
df_classification["meta_language"] = df_tests['meta'].apply(lambda x: x['language'])

# add meta source
df_classification["meta_source"] = df_tests['meta'].apply(lambda x: x['source'])

# add meta category 
df_classification["meta_category"] = df_tests['meta'].apply(lambda x: x.get("category", None))

### Prepare X and Y:

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
count_vect = CountVectorizer(max_features=150, lowercase=False)

In [10]:
X = df_classification[["type"]].copy().reset_index(drop=True)
X["type"] = pd.Categorical(df_classification["type"]).codes
X["meta_language"] = pd.Categorical(df_classification["meta_language"].fillna(-1)).codes
X["meta_category"] = pd.Categorical(df_classification["meta_category"].fillna(-1)).codes

df_words = pd.DataFrame(np.array(count_vect.fit_transform(df_classification["text"].values).toarray()))
X = pd.concat([X, df_words], axis=1)

y = df_classification[['id']].copy()

## LET US MERGE [17, 18, 19, 20] -> [17] AND [11, 12] -> 12.

In [11]:
# this ids have the same task type according to test sample:
y['id'] = y["id"].replace(18, 17).values
y['id'] = y["id"].replace(19, 17).values
y['id'] = y["id"].replace(20, 17).values

y['id'] = y["id"].replace(12, 11).values

### Model:

In [12]:
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict

In [13]:
clf = LGBMClassifier()

In [14]:
y_pred = cross_val_predict(clf, X, y.values.ravel(), cv=10, )

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

In [17]:
accuracy_score(y.values.ravel(), y_pred)

1.0